In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/domesticusairflight2016-2018/combine_files.csv


In [3]:
from copy import deepcopy
import json 

In [4]:
raw = pd.read_csv("/kaggle/input/domesticusairflight2016-2018/combine_files.csv", 
                  keep_default_na=False,
                  usecols=['Year',
                           'Origin',
                           'Dest',
                           'ActualElapsedTime'
                          ])

In [5]:
df = deepcopy(raw)
df.head()

,Year,ActualElapsedTime,Origin,Dest
0,2016,155.0,DFW,DTW
1,2016,150.0,DFW,DTW
2,2016,170.0,DFW,DTW
3,2016,151.0,DFW,DTW
4,2016,171.0,DFW,DTW


In [47]:
df_remain = df.sample(frac = 0.00006)

In [48]:
df_remain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1110 entries, 5684737 to 3692731
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Year               1110 non-null   int64 
 1   ActualElapsedTime  1110 non-null   object
 2   Origin             1110 non-null   object
 3   Dest               1110 non-null   object
dtypes: int64(1), object(3)
memory usage: 43.4+ KB


In [49]:
years = [2016, 2017, 2018]
dictionary = dict()

for year in years:
    dictionary[year] = {
        'origin': df_remain[df_remain['Year'] == year]['Origin'].to_numpy(),
        'dest': df_remain[df_remain['Year'] == year]['Dest'].to_numpy(),
        'origin_u': df_remain[df_remain['Year'] == year]['Origin'].unique(),
        'dest_u': df_remain[df_remain['Year'] == year]['Dest'].unique(),
        'actualtime': df_remain[df_remain['Year'] == year]['ActualElapsedTime'].to_numpy(),
    }

In [50]:
# 'origin': df[df['Year'] == year]['Origin'].to_numpy(),
#         'dest': df[df['Year'] == year]['Dest'].to_numpy(),
#         'origin_u': df[df['Year'] == year]['Origin'].unique(),
#         'dest_u': df[df['Year'] == year]['Dest'].unique(),
#         'distance': df[df['Year'] == year]['Distance'].to_numpy(),

# create new json file foreach year
for year in dictionary:
    origins = dictionary[year]['origin']
    destinations = dictionary[year]['dest']
    origin_unique = dictionary[year]['origin_u']
    dest_unique = dictionary[year]['dest_u']
    actualtimes = dictionary[year]['actualtime']
    cities = np.unique(np.concatenate((origin_unique, dest_unique),0))
    
    jsonfile = dict()
    
    # create nodes dictionary
    nodes = []

    for city in cities:
        nodes.append(
            {"name": city}
        )
    
    # create links dictionary
    links = []
    temp = None

    for origin, dest, timelength in zip(origins, destinations, actualtimes):
        if(temp != (origin, dest)):
            links.append(
                {"source": origin, "target": dest, "actualtime": timelength}
            )
            temp = (origin, dest)
            
    jsonfile["nodes"] = nodes
    jsonfile["links"] = links
    
    filename = str(year) + ".json"
    with open(filename, "w") as outfile:
        json.dump(jsonfile, outfile)